In [71]:
# Final Project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import csv
import random
%matplotlib inline
from numpy import genfromtxt
from keras import Sequential
from keras import callbacks
from keras.layers import Dense
from keras.initializers import glorot_normal
from keras.optimizers import SGD

# For tensorboard
tbCallBack = callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [72]:
# Loading data from the csv
airQuality = pd.read_csv("indice_2017Rev.csv")
airQuality

,Fecha,Hora,Ozono,Dioxido de azufre,Dioxido de nitrogeno,Monoxido de carbono,pm10,Zona
0,1/1/17,1,9.0,18.0,18.0,12.0,80.0,1
1,1/1/17,2,5.0,15.0,17.0,13.0,86.0,1
2,1/1/17,3,6.0,12.0,16.0,13.0,94.0,1
3,1/1/17,4,3.0,9.0,17.0,13.0,100.0,1
4,1/1/17,5,3.0,8.0,16.0,13.0,103.0,1
5,1/1/17,6,4.0,6.0,15.0,13.0,106.0,1
6,1/1/17,7,4.0,6.0,15.0,12.0,108.0,1
7,1/1/17,8,4.0,5.0,13.0,12.0,110.0,1
8,1/1/17,9,11.0,5.0,16.0,11.0,111.0,1
9,1/1/17,10,23.0,5.0,23.0,11.0,113.0,1


In [73]:
# Get the info from the dataset
airQuality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43800 entries, 0 to 43799
Data columns (total 8 columns):
Fecha                   43800 non-null object
Hora                    43800 non-null int64
Ozono                   42756 non-null float64
Dioxido de azufre       43782 non-null float64
Dioxido de nitrogeno    42741 non-null float64
Monoxido de carbono     41969 non-null float64
pm10                    42802 non-null float64
Zona                    43800 non-null int64
dtypes: float64(5), int64(2), object(1)
memory usage: 2.7+ MB


In [74]:
# Feature Engineering
# Checking impact of hour
airQuality[['Hora', 'Ozono']].groupby(['Hora'], as_index=False).mean()

,Hora,Ozono
0,1,16.321981
1,2,15.601974
2,3,15.033553
3,4,14.156495
4,5,12.858630
5,6,10.131507
6,7,7.658082
7,8,6.627945
8,9,9.212055
9,10,15.927671


In [75]:
# Filling empty Sulfur Dioxide rows with the median
so2Median = airQuality['Dioxido de azufre'].median()
airQuality['Dioxido de azufre'] = airQuality['Dioxido de azufre'].fillna(so2Median)
# Grouping in 4
airQuality['CategoricalSO2'] = pd.qcut(airQuality['Dioxido de azufre'], 4)
# Checking impact
airQuality[['CategoricalSO2', 'Ozono']].groupby(['CategoricalSO2'], as_index=False).mean()
# We can see that it does not have an impact as it Sulfur Dioxide is fairly uniform with respect of Ozone.
# Therefore, we may drop this feature.

,CategoricalSO2,Ozono
0,"(0.999, 2.0]",29.732082
1,"(2.0, 3.0]",32.455719
2,"(3.0, 6.0]",32.271811
3,"(6.0, 69.0]",29.208723


In [76]:
# Filling empty Nitrogen Dioxide rows with the median
no2Median = airQuality['Dioxido de nitrogeno'].median()
airQuality['Dioxido de nitrogeno'] = airQuality['Dioxido de nitrogeno'].fillna(no2Median)
# Grouping in 4
airQuality['CategoricalNO2'] = pd.qcut(airQuality['Dioxido de nitrogeno'], 4)
# Checking impact
airQuality[['CategoricalNO2', 'Ozono']].groupby(['CategoricalNO2'], as_index=False).mean()
# This feature does have some impact, we are going to keep this one.

,CategoricalNO2,Ozono
0,"(-0.001, 10.0]",42.081998
1,"(10.0, 15.0]",30.437596
2,"(15.0, 20.0]",24.425985
3,"(20.0, 62.0]",22.531719


In [77]:
# Filling empty Carbon Monoxide rows with the median
coMedian = airQuality['Monoxido de carbono'].median()
airQuality['Monoxido de carbono'] = airQuality['Monoxido de carbono'].fillna(coMedian)
# Grouping in 3
airQuality['CategoricalCO'] = pd.qcut(airQuality['Monoxido de carbono'], 4)
# Checking impact
airQuality[['CategoricalCO', 'Ozono']].groupby(['CategoricalCO'], as_index=False).mean()
# This feature has little changing impact, but may be important to keep it.

,CategoricalCO,Ozono
0,"(0.999, 5.0]",25.139447
1,"(5.0, 6.0]",26.846353
2,"(6.0, 8.0]",35.235867
3,"(8.0, 35.0]",38.132183


In [78]:
# Filling empty Nitrogen Dioxide rows with the median
pmMedian = airQuality['pm10'].median()
airQuality['pm10'] = airQuality['pm10'].fillna(pmMedian)
# Grouping in 3
airQuality['CategoricalPM'] = pd.qcut(airQuality['pm10'], 4)
# Checking impact
airQuality[['CategoricalPM', 'Ozono']].groupby(['CategoricalPM'], as_index=False).mean()
# It does have an impact on Ozone, we will keep it.

,CategoricalPM,Ozono
0,"(7.999, 44.0]",23.993164
1,"(44.0, 64.0]",31.909133
2,"(64.0, 94.0]",33.968850
3,"(94.0, 162.0]",32.640509


In [79]:
# Checking impact of Zone
airQuality[['Zona', 'Ozono']].groupby(['Zona'], as_index=False).mean()

,Zona,Ozono
0,1,27.599275
1,2,28.647059
2,3,28.330097
3,4,36.234304
4,5,31.818384


In [80]:
# Working dates: 24 measures by day
dayOfYear = np.full((43800, 1), 0)
number = 1
index = 0
for day in range(0,1825):
    for measure in range (0, 24):
        np.put(dayOfYear, index, number)
        index = index + 1
    number = number + 1
    if number == 366:
        number = 1
airQuality['DaysOfYear'] = dayOfYear
# Grouping by month (approx)
airQuality['CategoricalDays'] = pd.qcut(airQuality['DaysOfYear'], 12)
# Checking impact
airQuality[['CategoricalDays', 'Ozono']].groupby(['CategoricalDays'], as_index=False).mean()

,CategoricalDays,Ozono
0,"(0.999, 31.0]",30.065021
1,"(31.0, 61.0]",34.631669
2,"(61.0, 92.0]",30.992306
3,"(92.0, 122.0]",40.267236
4,"(122.0, 153.0]",46.997257
5,"(153.0, 183.0]",33.610556
6,"(183.0, 213.0]",25.279943
7,"(213.0, 244.0]",24.925377
8,"(244.0, 274.0]",21.924765
9,"(274.0, 305.0]",26.112742


In [81]:
# Filling Ozone's missing values
o3Median = airQuality['Ozono'].median()
airQuality['Ozono'] = airQuality['Ozono'].fillna(pmMedian)
# all filled and grouped!

In [82]:
# Mapping nitrogen dioxide
airQuality.loc[airQuality['Dioxido de nitrogeno'] <= 10.0, 'Dioxido de nitrogeno'] = 0
airQuality.loc[(airQuality['Dioxido de nitrogeno'] > 10.0) & (airQuality['Dioxido de nitrogeno'] <= 15.0), 'Dioxido de nitrogeno'] = 1
airQuality.loc[(airQuality['Dioxido de nitrogeno'] > 15.0) & (airQuality['Dioxido de nitrogeno'] <= 20.0), 'Dioxido de nitrogeno'] = 2
airQuality.loc[airQuality['Dioxido de nitrogeno'] > 20, 'Dioxido de nitrogeno'] = 3
airQuality['Dioxido de nitrogeno'] = airQuality['Dioxido de nitrogeno'].astype(int)

In [83]:
# Mapping Carbon Monoxide
airQuality.loc[airQuality['Monoxido de carbono'] <= 5, 'Monoxido de carbono'] = 0
airQuality.loc[(airQuality['Monoxido de carbono'] > 5) & (airQuality['Monoxido de carbono'] <= 6), 'Monoxido de carbono'] = 1
airQuality.loc[(airQuality['Monoxido de carbono'] > 6) & (airQuality['Monoxido de carbono'] <= 8), 'Monoxido de carbono'] = 2
airQuality.loc[airQuality['Monoxido de carbono'] > 8, 'Monoxido de carbono'] = 3
airQuality['Monoxido de carbono'] = airQuality['Monoxido de carbono'].astype(int)


In [84]:
# Mapping PM10
airQuality.loc[airQuality['pm10'] <= 44, 'pm10'] = 0
airQuality.loc[(airQuality['pm10'] > 44) & (airQuality['pm10'] <= 64), 'pm10'] = 1
airQuality.loc[(airQuality['pm10'] > 64) & (airQuality['pm10'] <= 94), 'pm10'] = 2
airQuality.loc[airQuality['pm10'] > 94, 'pm10'] = 3
airQuality['pm10'] = airQuality['pm10'].astype(int)

In [85]:
# Mapping days
airQuality.loc[airQuality['DaysOfYear'] <= 31, 'DaysOfYear'] = 0
airQuality.loc[(airQuality['DaysOfYear'] > 31) & (airQuality['DaysOfYear'] <= 61), 'DaysOfYear'] = 1
airQuality.loc[(airQuality['DaysOfYear'] > 61) & (airQuality['DaysOfYear'] <= 92), 'DaysOfYear'] = 2
airQuality.loc[(airQuality['DaysOfYear'] > 92) & (airQuality['DaysOfYear'] <= 122), 'DaysOfYear'] = 3
airQuality.loc[(airQuality['DaysOfYear'] > 122) & (airQuality['DaysOfYear'] <= 153), 'DaysOfYear'] = 4
airQuality.loc[(airQuality['DaysOfYear'] > 153) & (airQuality['DaysOfYear'] <= 183), 'DaysOfYear'] = 5
airQuality.loc[(airQuality['DaysOfYear'] > 183) & (airQuality['DaysOfYear'] <= 213), 'DaysOfYear'] = 6
airQuality.loc[(airQuality['DaysOfYear'] > 213) & (airQuality['DaysOfYear'] <= 244), 'DaysOfYear'] = 7
airQuality.loc[(airQuality['DaysOfYear'] > 244) & (airQuality['DaysOfYear'] <= 274), 'DaysOfYear'] = 8
airQuality.loc[(airQuality['DaysOfYear'] > 274) & (airQuality['DaysOfYear'] <= 305), 'DaysOfYear'] = 9
airQuality.loc[(airQuality['DaysOfYear'] > 305) & (airQuality['DaysOfYear'] <= 335), 'DaysOfYear'] = 10
airQuality.loc[airQuality['DaysOfYear'] > 335, 'DaysOfYear'] = 11
airQuality['DaysOfYear'] = airQuality['DaysOfYear'].astype(int)

In [86]:
dropFeat = ['Fecha', 'Dioxido de azufre', 'CategoricalSO2', 'CategoricalNO2', 'CategoricalCO', 'CategoricalPM', 'CategoricalDays']
airQuality = airQuality.drop(dropFeat, axis=1)

results = np.array(airQuality)

X = results[:, [0, 2, 3, 4, 5, 6]]
Y = results[:, [1]]

# Normalization
X = X / np.linalg.norm(X)
Y = Y / np.linalg.norm(Y)
# Splitting data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=73)

In [89]:
out_dimension = 1
nHidden = 4
nInputDimension = X_train.shape[1]

# Building the model and then running it
model = Sequential()
model.add(Dense(nHidden, input_dim = nInputDimension, activation = 'relu', kernel_initializer = glorot_normal(seed=0)))
model.add(Dense(out_dimension, activation = 'sigmoid', kernel_initializer = glorot_normal(seed=0)))

sgd = SGD(lr=0.05, decay=1e-6, momentum=0.5, nesterov=True) # optimizer

model.compile(loss='mean_squared_logarithmic_error', optimizer=sgd, metrics=['mae'])

# Last parameter (callbacks) is important for TensorBoard
train = model.fit(X_train, Y_train, epochs=100, callbacks=[tbCallBack])

# Evaluating model
score = model.evaluate(X_test, Y_test, batch_size=25)
score

Epoch 1/100
32850/32850 [==============================] - 1s 41us/step - loss: 0.0152 - mean_absolute_error: 0.1022
Epoch 2/100
32850/32850 [==============================] - 1s 33us/step - loss: 6.2050e-04 - mean_absolute_error: 0.0248
Epoch 3/100
32850/32850 [==============================] - 1s 33us/step - loss: 2.6685e-04 - mean_absolute_error: 0.0161
Epoch 4/100
32850/32850 [==============================] - 1s 32us/step - loss: 1.5945e-04 - mean_absolute_error: 0.0123
Epoch 5/100
32850/32850 [==============================] - 1s 33us/step - loss: 1.1012e-04 - mean_absolute_error: 0.0101
Epoch 6/100
32850/32850 [==============================] - 2s 67us/step - loss: 8.2642e-05 - mean_absolute_error: 0.0087
Epoch 7/100
32850/32850 [==============================] - 1s 33us/step - loss: 6.5504e-05 - mean_absolute_error: 0.0077
Epoch 8/100
32850/32850 [==============================] - 2s 46us/step - loss: 5.3976e-05 - mean_absolute_error: 0.0069
Epoch 9/100
32850/32850 [===========

32850/32850 [==============================] - 1s 38us/step - loss: 1.0764e-05 - mean_absolute_error: 0.0027
Epoch 66/100
32850/32850 [==============================] - 1s 34us/step - loss: 1.0742e-05 - mean_absolute_error: 0.0027
Epoch 67/100
32850/32850 [==============================] - 1s 38us/step - loss: 1.0722e-05 - mean_absolute_error: 0.0027
Epoch 68/100
32850/32850 [==============================] - 1s 37us/step - loss: 1.0702e-05 - mean_absolute_error: 0.0027
Epoch 69/100
32850/32850 [==============================] - 1s 43us/step - loss: 1.0683e-05 - mean_absolute_error: 0.0026
Epoch 70/100
32850/32850 [==============================] - 1s 38us/step - loss: 1.0666e-05 - mean_absolute_error: 0.0026: 0s - loss: 1.0830e-05 - mean_absolute_error: 0
Epoch 71/100
32850/32850 [==============================] - 1s 38us/step - loss: 1.0649e-05 - mean_absolute_error: 0.0026: 0s - loss: 1.0656e-05 - mean_absolute_er
Epoch 72/100
32850/32850 [==============================] - 1s 40us/s

[1.0487231903118392e-05, 0.0025252502474861567]

In [88]:
# To see the model's plotting: tensorboard --logdir ./Graph 
